In [1]:
import numpy as np
import pandas as pd
import sklearn
from zipfile import ZipFile

### Reading in the Dataset

In [2]:
zip_file = ZipFile('census_data.zip')
df = pd.read_csv(zip_file.open('census_data.csv'))

Inspecting a small sample of the data frame

In [3]:
df.head(n = 5)

,age,class_of_worker,education,wage_per_hour,marital_status,major_industry_code,major_occupation_code,race,sex,full_or_part_time_employment_stat,...,live_in_this_house_1_year_ago,family_members_under,total_person_earnings,country_of_birth_father,country_of_birth_mother,country_of_birth_self,citizenship,own_business_or_self_employed,weeks_worked,earn_over_50k
0,73,Not in universe,High school graduate,0,Widowed,Not in universe or children,Not in universe,White,Female,Not in labor force,...,Not in universe under 1 year old,0,Not in universe,United-States,United-States,United-States,Native- Born in the United States,Not in universe,0,-50000
1,58,Self-employed-not incorporated,Some college but no degree,0,Divorced,Construction,Precision production craft & repair,White,Male,Children or Armed Forces,...,No,1,Not in universe,United-States,United-States,United-States,Native- Born in the United States,Not in universe,52,-50000
2,18,Not in universe,10th grade,0,Never married,Not in universe or children,Not in universe,Asian or Pacific Islander,Female,Not in labor force,...,Not in universe under 1 year old,0,Not in universe,Vietnam,Vietnam,Vietnam,Foreign born- Not a citizen of U S,Not in universe,0,-50000
3,9,Not in universe,Children,0,Never married,Not in universe or children,Not in universe,White,Female,Children or Armed Forces,...,Yes,0,Both parents present,United-States,United-States,United-States,Native- Born in the United States,Not in universe,0,-50000
4,10,Not in universe,Children,0,Never married,Not in universe or children,Not in universe,White,Female,Children or Armed Forces,...,Yes,0,Both parents present,United-States,United-States,United-States,Native- Born in the United States,Not in universe,0,-50000


### Exploring and Cleaning the Data

In [6]:
df.describe()

,age,wage_per_hour,capital_gains,capital_losses,divdends_from_stocks,family_members_under,weeks_worked
count,199523.000000,199523.000000,199523.00000,199523.000000,199523.000000,199523.000000,199523.000000
mean,34.494199,55.426908,434.71899,37.313788,197.529533,1.956180,23.174897
std,22.310895,274.896454,4697.53128,271.896428,1984.163658,2.365126,24.411488
min,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
25%,15.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
50%,33.000000,0.000000,0.00000,0.000000,0.000000,1.000000,8.000000
75%,50.000000,0.000000,0.00000,0.000000,0.000000,4.000000,52.000000
max,90.000000,9999.000000,99999.00000,4608.000000,99999.000000,6.000000,52.000000


In [19]:
df.dtypes

age                                   int64
class_of_worker                      object
education                            object
wage_per_hour                         int64
marital_status                       object
major_industry_code                  object
major_occupation_code                object
race                                 object
sex                                  object
full_or_part_time_employment_stat    object
capital_gains                         int64
capital_losses                        int64
divdends_from_stocks                  int64
live_in_this_house_1_year_ago        object
family_members_under                  int64
total_person_earnings                object
country_of_birth_father              object
country_of_birth_mother              object
country_of_birth_self                object
citizenship                          object
own_business_or_self_employed        object
weeks_worked                          int64
earn_over_50k                   

Writing a function which finds columns by type

In [33]:
def get_cols_by_type(dataframe, dtype):
    '''
    Takes a dataframe and dtype, and returns a list of columns names of that dtype.
    '''
    index = np.where(df.dtypes == dtype)[0]
    print(str(dtype) + ":")
    print(list(dataframe.columns[index]))
    return list(dataframe.columns[index])
    

In [36]:
obj_cols = get_cols_by_type(df,'object')
int_cols = get_cols_by_type(df, 'int64')

object:
['class_of_worker', 'education', 'marital_status', 'major_industry_code', 'major_occupation_code', 'race', 'sex', 'full_or_part_time_employment_stat', 'live_in_this_house_1_year_ago', 'total_person_earnings', 'country_of_birth_father', 'country_of_birth_mother', 'country_of_birth_self', 'citizenship', 'own_business_or_self_employed', 'earn_over_50k']
int64:
['age', 'wage_per_hour', 'capital_gains', 'capital_losses', 'divdends_from_stocks', 'family_members_under', 'weeks_worked']


Now we go through each feature (column) and decide which category of data it belongs to. These categories are nominal, ordinal, interval, ratio, target

In [37]:
nominal_cols = [ 'sex', 'race', 'marital_status', 'major_industry_code', 'major_occupation_code', 'full_or_part_time_employment_stat',  \
               'country_of_birth_father', 'country_of_birth_mother', 'country_of_birth_self', 'live_in_this_house_1_year_ago', 'own_business_or_self_employed',\
                'total_person_earnings', 'citizenship']

ordinal_cols = [ 'education', 'class_of_worker']

interval_cols = [] 

ratio_cols = ['age', 'family_members_under', 'weeks_worked', 'wage_per_hour', 'capital_gains', 'capital_losses', 'divdends_from_stocks' ]

target_col = ["earn_over_50k"]

Lets look at the possible values that the nominal columns can take. This will allow us to remove some unwanted data.

In [38]:
for feature in nominal_cols:
    print(feature+":")
    print(sorted(df[feature].unique()))
    print("")

sex:
['Female', 'Male']

race:
['Amer Indian Aleut or Eskimo', 'Asian or Pacific Islander', 'Black', 'Other', 'White']

marital_status:
['Divorced', 'Married-A F spouse present', 'Married-civilian spouse present', 'Married-spouse absent', 'Never married', 'Separated', 'Widowed']

major_industry_code:
['Agriculture', 'Armed Forces', 'Business and repair services', 'Communications', 'Construction', 'Education', 'Entertainment', 'Finance insurance and real estate', 'Forestry and fisheries', 'Hospital services', 'Manufacturing-durable goods', 'Manufacturing-nondurable goods', 'Medical except hospital', 'Mining', 'Not in universe or children', 'Other professional services', 'Personal services except private HH', 'Private household services', 'Public administration', 'Retail trade', 'Social services', 'Transportation', 'Utilities and sanitary services', 'Wholesale trade']

major_occupation_code:
['Adm support including clerical', 'Armed Forces', 'Executive admin and managerial', 'Farming for

We can see that there are some '?' data points which are undesirable. It makes sense to use the modal data point for that column to replace these '?' values.

In [ ]:
for feature in ["country_of_birth_father", "country_of_birth_mother", "country_of_birth_self"]:
    df[feature] = df[feature].replace(['?'], df[feature].mode()[0])

Re-affirming that these are in fact removed from the data sets

In [45]:
for feature in nominal_cols:
    print(feature+":")
    print(sorted(df[feature].unique()))
    print("")

sex:
['Female', 'Male']

race:
['Amer Indian Aleut or Eskimo', 'Asian or Pacific Islander', 'Black', 'Other', 'White']

marital_status:
['Divorced', 'Married-A F spouse present', 'Married-civilian spouse present', 'Married-spouse absent', 'Never married', 'Separated', 'Widowed']

major_industry_code:
['Agriculture', 'Armed Forces', 'Business and repair services', 'Communications', 'Construction', 'Education', 'Entertainment', 'Finance insurance and real estate', 'Forestry and fisheries', 'Hospital services', 'Manufacturing-durable goods', 'Manufacturing-nondurable goods', 'Medical except hospital', 'Mining', 'Not in universe or children', 'Other professional services', 'Personal services except private HH', 'Private household services', 'Public administration', 'Retail trade', 'Social services', 'Transportation', 'Utilities and sanitary services', 'Wholesale trade']

major_occupation_code:
['Adm support including clerical', 'Armed Forces', 'Executive admin and managerial', 'Farming for

We also notice that there are lots of 'Not in universe' values. We shall change this (and variations of this value) to 'null'.

In [46]:
df = df.replace("Not in universe","null").replace("Not in universe or children", "null")\
                                            .replace("Not in universe under 1 year old", "null")

Checking to see that the null values have been implemented

In [47]:
for feature in nominal_cols:
    print(feature+":")
    print(sorted(df[feature].unique()))
    print("")

sex:
['Female', 'Male']

race:
['Amer Indian Aleut or Eskimo', 'Asian or Pacific Islander', 'Black', 'Other', 'White']

marital_status:
['Divorced', 'Married-A F spouse present', 'Married-civilian spouse present', 'Married-spouse absent', 'Never married', 'Separated', 'Widowed']

major_industry_code:
['Agriculture', 'Armed Forces', 'Business and repair services', 'Communications', 'Construction', 'Education', 'Entertainment', 'Finance insurance and real estate', 'Forestry and fisheries', 'Hospital services', 'Manufacturing-durable goods', 'Manufacturing-nondurable goods', 'Medical except hospital', 'Mining', 'Other professional services', 'Personal services except private HH', 'Private household services', 'Public administration', 'Retail trade', 'Social services', 'Transportation', 'Utilities and sanitary services', 'Wholesale trade', 'null']

major_occupation_code:
['Adm support including clerical', 'Armed Forces', 'Executive admin and managerial', 'Farming forestry and fishing', 'Ha

### Generating our Features

#### Nominal Columns

I am going to binarise the following columns such that its easier to build the model. We have previously seen that the 'United States' is the most common country - therefore for country features, we will set 'United States' to be 1, and the other countries to be 0. For sex, male = 1 and female = 0 so on and so forth.

We will create new dataframe columns to house this data.

In [48]:
features = [ 'sex', 'live_in_this_house_1_year_ago', 'country_of_birth_father', 'country_of_birth_mother',\
            'country_of_birth_self', 'own_business_or_self_employed' ]
df['binary_sex'] = np.where(df['sex'] == 'Male',1,0)
df['binary_LITH1YA'] = np.where(df['live_in_this_house_1_year_ago'] == 'Yes',1,0)
df['binary_father_birth'] = np.where(df['country_of_birth_father'] == 'United-States',1,0)
df['binary_mother_birth'] = np.where(df['country_of_birth_mother'] == 'United-States',1,0)
df['binary_self_birth'] = np.where(df['country_of_birth_self'] == 'United-States',1,0)
df['binary_OB_SE'] = np.where(df['own_business_or_self_employed'] == 'Yes',1,0)

In [49]:
df.head()

,age,class_of_worker,education,wage_per_hour,marital_status,major_industry_code,major_occupation_code,race,sex,full_or_part_time_employment_stat,...,citizenship,own_business_or_self_employed,weeks_worked,earn_over_50k,binary_sex,binary_LITH1YA,binary_father_birth,binary_mother_birth,binary_self_birth,binary_OB_SE
0,73,null,High school graduate,0,Widowed,null,null,White,Female,Not in labor force,...,Native- Born in the United States,null,0,-50000,0,0,1,1,1,0
1,58,Self-employed-not incorporated,Some college but no degree,0,Divorced,Construction,Precision production craft & repair,White,Male,Children or Armed Forces,...,Native- Born in the United States,null,52,-50000,1,0,1,1,1,0
2,18,null,10th grade,0,Never married,null,null,Asian or Pacific Islander,Female,Not in labor force,...,Foreign born- Not a citizen of U S,null,0,-50000,0,0,0,0,0,0
3,9,null,Children,0,Never married,null,null,White,Female,Children or Armed Forces,...,Native- Born in the United States,null,0,-50000,0,1,1,1,1,0
4,10,null,Children,0,Never married,null,null,White,Female,Children or Armed Forces,...,Native- Born in the United States,null,0,-50000,0,1,1,1,1,0


Now we have created new columns, we can drop the 'parent' columns that the binary columns were derived from

In [ ]:
drop_cols = ["sex", "live_in_this_house_1_year_ago", "country_of_birth_father",
            "country_of_birth_mother", "country_of_birth_self", "own_business_or_self_employed"]
df = df.drop(columns = drop_cols)

In [50]:
df.head()

,age,class_of_worker,education,wage_per_hour,marital_status,major_industry_code,major_occupation_code,race,sex,full_or_part_time_employment_stat,...,citizenship,own_business_or_self_employed,weeks_worked,earn_over_50k,binary_sex,binary_LITH1YA,binary_father_birth,binary_mother_birth,binary_self_birth,binary_OB_SE
0,73,null,High school graduate,0,Widowed,null,null,White,Female,Not in labor force,...,Native- Born in the United States,null,0,-50000,0,0,1,1,1,0
1,58,Self-employed-not incorporated,Some college but no degree,0,Divorced,Construction,Precision production craft & repair,White,Male,Children or Armed Forces,...,Native- Born in the United States,null,52,-50000,1,0,1,1,1,0
2,18,null,10th grade,0,Never married,null,null,Asian or Pacific Islander,Female,Not in labor force,...,Foreign born- Not a citizen of U S,null,0,-50000,0,0,0,0,0,0
3,9,null,Children,0,Never married,null,null,White,Female,Children or Armed Forces,...,Native- Born in the United States,null,0,-50000,0,1,1,1,1,0
4,10,null,Children,0,Never married,null,null,White,Female,Children or Armed Forces,...,Native- Born in the United States,null,0,-50000,0,1,1,1,1,0


#### Ordinal Columns